In [1]:
import os 

In [2]:
current_directory = os.getcwd()
print(current_directory)

c:\Users\kakde\OneDrive\Desktop\Drowsiness Detection using Deep Learning\research


In [3]:
os.chdir('../')

In [4]:
current_directory = os.getcwd()
print(current_directory)

c:\Users\kakde\OneDrive\Desktop\Drowsiness Detection using Deep Learning


In [5]:
from collections import namedtuple
import os

In [6]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-22 13:28:25,220: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-22 13:28:25,224: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-22 13:28:25,228: INFO: common: created directory at: artifacts]
[2024-12-22 13:28:25,231: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-22 13:29:58,137: INFO: 2896332844: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 115969251
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'
Content-Type: application/zip
ETag: "16e92400c1b08d7c829256f0bd7988a29b6c9b3cb90738645d109f9116b5dfde"
Last-Modified: Sun, 22 Dec 2024 07:58:28 GMT
Strict-Transport-Security: max-age=31557600
timing-allow-origin: https://github.com
X-Content-Type-Options: nosniff
X-Frame-Options: deny
x-github-tenant: 
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: ED40:25B585:2ED5C0:44E7F1:6767C6A3
Accept-Ranges: bytes
Date: Sun, 22 Dec 2024 07: